# Proyecto 2: Obteniendo los data frames con los que trabajaremos
Curso Ingeniería de Características

**Estephania Pivac Alcaraz**

**Profesor: Dr. Julio Waissman**

---

**Problema de interés:**
Estudiar la relación que existe entre la cantidad de accidentes de tránsito y la inversión a la conservación de carreteras y presupuesto gubernamental.

## 1. Abriendo las bases de datos

Abrimos las bases de datos que estaremos utilizando. Para esto construiremos los dataframes necesarios así como también limpiaremos y analizaremos dichos datos.

Comenzamos importando librerías que necesitaremos

In [45]:
# Instalamos Pandas-profiling
#pip install Pandas-Profiling

# Importamos algunas librerías que vamos a necesitar
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos
import numpy as np
from pandas_profiling import ProfileReport

### 1.1 Tidy_accidentes: Base de datos de accidentes de tránsito

Abrimos el archivo con la información de los accidentes de tránsito en México desde 2017 hasta 2021. Generamos el dataframe con el que estaremos trabajando y convertimos cada columna al tipo de dato adecuado.

In [46]:
# Generamos el dataframe de accidentes
tidy_accidentes = pd.read_csv("tidy_accidentes.csv", index_col=False)

# Convertimos cada variable de acuerdo a su tipo
# Categoricas
#tidy_accidentes = tidy_accidentes.astype({'TIPACCID':'category', 'CAUSAACCI':'category', 'CAPAROD':'category', 'CLASACC':'category', 'EDAD':'category'})

# Fechas
tidy_accidentes['FECHA'] = pd.to_datetime(tidy_accidentes['FECHA'], format='%Y/%m/%d', errors='coerce')
#tidy_accidentes.info()

Observemos la estructura de la base de datos:

In [47]:
tidy_accidentes.head()

,ID_ENTIDAD,ANIO,TIPACCID,CAUSAACCI,CAPAROD,CLASACC,FECHA,EDAD
0,1,2021,Salida del camino,Conductor,Pavimentada,Sólo daños,2021-01-03,28-37
1,1,2021,Salida del camino,Conductor,Pavimentada,No fatal,2021-01-04,Menor de edad
2,1,2021,Colisión con vehículo automotor,Conductor,Pavimentada,Sólo daños,2021-01-17,48-57
3,1,2021,Salida del camino,Conductor,Pavimentada,Sólo daños,2021-01-26,Se fugó
4,1,2021,Colisión con vehículo automotor,Conductor,Pavimentada,Sólo daños,2021-01-28,38-47


Veamos que esta base de datos no contiene datos faltantes:

In [48]:
tidy_accidentes.isnull().sum()

ID_ENTIDAD    0
ANIO          0
TIPACCID      0
CAUSAACCI     0
CAPAROD       0
CLASACC       0
FECHA         0
EDAD          0
dtype: int64

Seleccionamos únicamente las columnas de interés:

In [49]:
tidy_accidentes = tidy_accidentes[['ID_ENTIDAD', 'ANIO', 'CAUSAACCI', 'CLASACC']]
tidy_accidentes

,ID_ENTIDAD,ANIO,CAUSAACCI,CLASACC
0,1,2021,Conductor,Sólo daños
1,1,2021,Conductor,No fatal
2,1,2021,Conductor,Sólo daños
3,1,2021,Conductor,Sólo daños
4,1,2021,Conductor,Sólo daños
...,...,...,...,...
81843,32,2017,Conductor,Sólo daños
81844,32,2017,Conductor,Fatal
81845,32,2017,Conductor,Fatal
81846,32,2017,Conductor,No fatal


### 1.2 Budget transparency: Base de datos sobre el presupuesto público aprobado de 2013 a 2021

In [50]:
# Generamos el dataframe de presupuesto
df_budget = pd.read_csv("budget_transparency.csv", index_col=False)
df_budget

# Convertimos cada variable de acuerdo a su tipo
# Categoricas
df_budget = df_budget.astype({'State':'category', 'Expenses Type ID':'category'})

# Convertimos Year a datetime
df_budget['Year'] = pd.to_datetime(df_budget['Year'], format='%Y', errors='coerce')

df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   State ID          288 non-null    int64         
 1   State             288 non-null    category      
 2   Year              288 non-null    datetime64[ns]
 3   Expenses Type ID  288 non-null    category      
 4   Expenses Type     288 non-null    object        
 5   Amount Approved   288 non-null    int64         
dtypes: category(2), datetime64[ns](1), int64(2), object(1)
memory usage: 11.1+ KB


In [51]:
# Exploramos el dataframe
df_budget

,State ID,State,Year,Expenses Type ID,Expenses Type,Amount Approved
0,1,Aguascalientes,2013-01-01,8,Gasto de Obra Pública,1511908040
1,1,Aguascalientes,2014-01-01,8,Gasto de Obra Pública,1865135779
2,1,Aguascalientes,2015-01-01,8,Gasto de Obra Pública,1659723400
3,1,Aguascalientes,2016-01-01,8,Gasto de Obra Pública,1717532284
4,1,Aguascalientes,2017-01-01,8,Gasto de Obra Pública,1230564370
...,...,...,...,...,...,...
283,32,Zacatecas,2017-01-01,8,Gasto de Obra Pública,1992505733
284,32,Zacatecas,2018-01-01,8,Gasto de Obra Pública,2284882371
285,32,Zacatecas,2019-01-01,8,Gasto de Obra Pública,2567673138
286,32,Zacatecas,2020-01-01,8,Gasto de Obra Pública,2010497486


In [52]:
# Observamos si existen datos faltantes
df_budget.isnull().sum()

State ID            0
State               0
Year                0
Expenses Type ID    0
Expenses Type       0
Amount Approved     0
dtype: int64

In [53]:
df_budget = df_budget[['State ID', 'State', 'Year', 'Amount Approved']]
df_budget

,State ID,State,Year,Amount Approved
0,1,Aguascalientes,2013-01-01,1511908040
1,1,Aguascalientes,2014-01-01,1865135779
2,1,Aguascalientes,2015-01-01,1659723400
3,1,Aguascalientes,2016-01-01,1717532284
4,1,Aguascalientes,2017-01-01,1230564370
...,...,...,...,...
283,32,Zacatecas,2017-01-01,1992505733
284,32,Zacatecas,2018-01-01,2284882371
285,32,Zacatecas,2019-01-01,2567673138
286,32,Zacatecas,2020-01-01,2010497486


## 2. Obteniendo el dataframe tidy_suma_estatal

Juntaremos las dos bases de datos con las que trabajaremos 

### 2.1 Obtener las sumas de accidentes.

In [54]:
tidy_accidentes.head()

,ID_ENTIDAD,ANIO,CAUSAACCI,CLASACC
0,1,2021,Conductor,Sólo daños
1,1,2021,Conductor,No fatal
2,1,2021,Conductor,Sólo daños
3,1,2021,Conductor,Sólo daños
4,1,2021,Conductor,Sólo daños


In [55]:
df_budget.head()

,State ID,State,Year,Amount Approved
0,1,Aguascalientes,2013-01-01,1511908040
1,1,Aguascalientes,2014-01-01,1865135779
2,1,Aguascalientes,2015-01-01,1659723400
3,1,Aguascalientes,2016-01-01,1717532284
4,1,Aguascalientes,2017-01-01,1230564370


In [56]:
# Utilizaremos onehot encoding para generar variables dummies

# Convertimos las variables categoricas al tipo adecuado
df_accidentes = tidy_accidentes.astype({'CAUSAACCI':'category', 'CLASACC':'category'})

# Creamos una copia del dataframe tidy accidentes
df_accidentes_onehot = df_accidentes.copy()

# Agregamos una nueva columna al dataframe que identifique cada accidente con 1.
df_accidentes_onehot['Total_accidentes'] = 1

# Identificamos las variables cualitativas
variables_cualitativas = ['CAUSAACCI', 'CLASACC']

# Aplicamos el método OneHot:
for x in variables_cualitativas:
  df_accidentes_onehot = pd.get_dummies(
      df_accidentes_onehot, 
      columns = [x], 
      prefix = [x],
      dtype=int
  )

# Obtenemos nuestro dataframe
df_accidentes_onehot

,ID_ENTIDAD,ANIO,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños
0,1,2021,1,1,0,0,0,0,1
1,1,2021,1,1,0,0,0,1,0
2,1,2021,1,1,0,0,0,0,1
3,1,2021,1,1,0,0,0,0,1
4,1,2021,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
81843,32,2017,1,1,0,0,0,0,1
81844,32,2017,1,1,0,0,1,0,0
81845,32,2017,1,1,0,0,1,0,0
81846,32,2017,1,1,0,0,0,1,0


In [57]:
# Obtenemos los nombres de las columnas de nuestro dataframe
nombres_columnas = list(df_accidentes_onehot.columns.values)

# Creamos un nuevo dataframe con las mismas columnas
tidy_suma_est = pd.DataFrame(columns = nombres_columnas) 

# Sumamos el total de accidentes por año y por entidad
for j in range(1, 32):
    for i in range(2017, 2022):
        df = pd.DataFrame(df_accidentes_onehot[df_accidentes_onehot['ANIO'] == i].iloc[:, 1:].sum(axis=0, numeric_only=True))
        df = df.transpose()
        df['ANIO'] = i
        df['ID_ENTIDAD'] = j
        tidy_suma_est = tidy_suma_est.append(df)

# Reinicamos los índices
tidy_suma_est = tidy_suma_est.reset_index(drop = True)

# Obtenemos nuestro dataframe
tidy_suma_est = tidy_suma_est.astype(int)
tidy_suma_est

,ID_ENTIDAD,ANIO,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños
0,1,2017,14966,14272,362,332,894,4102,9970
1,1,2018,17592,16726,480,386,914,4431,12247
2,1,2019,16757,16075,516,166,837,4509,11411
3,1,2020,15057,14448,441,168,775,3971,10311
4,1,2021,17476,16738,513,225,859,4741,11876
...,...,...,...,...,...,...,...,...,...
150,31,2017,14966,14272,362,332,894,4102,9970
151,31,2018,17592,16726,480,386,914,4431,12247
152,31,2019,16757,16075,516,166,837,4509,11411
153,31,2020,15057,14448,441,168,775,3971,10311


### 2.2 Juntamos los dataframes

In [58]:
# Para facilitar el uso de la función join agregamos una columna con solo la información del año
c = []
for i in range(0, len(df_budget)):
    y = df_budget.loc[i, 'Year'].year
    c.append(y)

df_budget['ANIO'] = c

# Creamos una copia del dataframe df_budget
df_budget_sum = df_budget.copy()
df_budget_sum['ANIO'] = df_budget_sum['ANIO'].astype(str)
df_budget_sum

/var/folders/r2/wrdzf10d2lq1sjgp521xsvrm0000gn/T/ipykernel_10689/2301010147.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_budget['ANIO'] = c


,State ID,State,Year,Amount Approved,ANIO
0,1,Aguascalientes,2013-01-01,1511908040,2013
1,1,Aguascalientes,2014-01-01,1865135779,2014
2,1,Aguascalientes,2015-01-01,1659723400,2015
3,1,Aguascalientes,2016-01-01,1717532284,2016
4,1,Aguascalientes,2017-01-01,1230564370,2017
...,...,...,...,...,...
283,32,Zacatecas,2017-01-01,1992505733,2017
284,32,Zacatecas,2018-01-01,2284882371,2018
285,32,Zacatecas,2019-01-01,2567673138,2019
286,32,Zacatecas,2020-01-01,2010497486,2020


Agregamos la información sobre el presupuesto estatal aprobado anualmente en Sonora.

In [59]:
nc = []
sc = []
for j in range(0, len(tidy_suma_est)):
    for i in range(0, len(df_budget)):
        if ((df_budget['State ID'][i] == tidy_suma_est['ID_ENTIDAD'][j]) and (df_budget['ANIO'][i] == tidy_suma_est['ANIO'][j])):
            nc.append(df_budget['Amount Approved'][i])
            sc.append(df_budget['State'][i])
nc           
tidy_suma_est['Presupuesto'] = nc
tidy_suma_est['Entidad'] = sc
tidy_suma_est

,ID_ENTIDAD,ANIO,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,Presupuesto,Entidad
0,1,2017,14966,14272,362,332,894,4102,9970,1230564370,Aguascalientes
1,1,2018,17592,16726,480,386,914,4431,12247,763497156,Aguascalientes
2,1,2019,16757,16075,516,166,837,4509,11411,698139978,Aguascalientes
3,1,2020,15057,14448,441,168,775,3971,10311,801262825,Aguascalientes
4,1,2021,17476,16738,513,225,859,4741,11876,619602783,Aguascalientes
...,...,...,...,...,...,...,...,...,...,...,...
150,31,2017,14966,14272,362,332,894,4102,9970,4083206552,Yucatán
151,31,2018,17592,16726,480,386,914,4431,12247,3096657566,Yucatán
152,31,2019,16757,16075,516,166,837,4509,11411,3064347028,Yucatán
153,31,2020,15057,14448,441,168,775,3971,10311,2308676495,Yucatán


### 2.3 Guardamos el dataframe tidy_suma_anual como csv

In [60]:
# Guardamos el data frame en un csv
tidy_suma_est.to_csv('tidy_suma_anual.csv', header=True, index=False)

## 3. Obteniendo el dataframe tidy_suma

### 3.1 Obtener las sumas de accidentes de acuerdo a las categorías de interés

In [61]:
# Utilizaremos onehot encoding para generar variables dummies

# Convertimos las variables categoricas al tipo adecuado
df_accidentes = tidy_accidentes.astype({'CAUSAACCI':'category', 'CLASACC':'category'})

# Creamos una copia del dataframe tidy accidentes
df_accidentes_onehot = df_accidentes.copy()

# Agregamos una nueva columna al dataframe que identifique cada accidente con 1.
df_accidentes_onehot['Total_accidentes'] = 1

# Identificamos las variables cualitativas
variables_cualitativas = ['CAUSAACCI', 'CLASACC', 'ANIO']

# Aplicamos el método OneHot:
for x in variables_cualitativas:
  df_accidentes_onehot = pd.get_dummies(
      df_accidentes_onehot, 
      columns = [x], 
      prefix = [x],
      dtype=int
  )

# Obtenemos nuestro dataframe
df_accidentes_onehot

,ID_ENTIDAD,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,ANIO_2017,ANIO_2018,ANIO_2019,ANIO_2020,ANIO_2021
0,1,1,1,0,0,0,0,1,0,0,0,0,1
1,1,1,1,0,0,0,1,0,0,0,0,0,1
2,1,1,1,0,0,0,0,1,0,0,0,0,1
3,1,1,1,0,0,0,0,1,0,0,0,0,1
4,1,1,1,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81843,32,1,1,0,0,0,0,1,1,0,0,0,0
81844,32,1,1,0,0,1,0,0,1,0,0,0,0
81845,32,1,1,0,0,1,0,0,1,0,0,0,0
81846,32,1,1,0,0,0,1,0,1,0,0,0,0


In [62]:
# Obtenemos los nombres de las columnas de nuestro dataframe
nombres_columnas = list(df_accidentes_onehot.columns.values)

# Creamos un nuevo dataframe con las mismas columnas
tidy_suma = pd.DataFrame(columns = nombres_columnas) 

# Sumamos el total de accidentes por año y por entidad
for j in range(1, 32):
    df = pd.DataFrame(df_accidentes_onehot[df_accidentes_onehot['ID_ENTIDAD'] == j].iloc[:, 1:].sum(axis=0, numeric_only=True))
    df = df.transpose()
    df['ID_ENTIDAD'] = j
    tidy_suma = tidy_suma.append(df)

# Reinicamos los índices
tidy_suma = tidy_suma.reset_index(drop = True)

# Obtenemos nuestro dataframe
tidy_suma = tidy_suma.astype(int)
tidy_suma

,ID_ENTIDAD,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,ANIO_2017,ANIO_2018,ANIO_2019,ANIO_2020,ANIO_2021
0,1,1222,1192,30,0,86,440,696,228,241,191,254,308
1,2,489,479,1,9,19,128,342,14,60,74,123,218
2,3,93,87,5,1,4,30,59,21,21,18,15,18
3,4,559,505,50,4,52,182,325,117,126,121,97,98
4,5,466,427,33,6,30,88,348,71,69,105,77,144
5,6,1852,1655,142,55,42,650,1160,365,360,355,387,385
6,7,2492,2485,5,2,214,674,1604,415,457,613,448,559
7,8,3674,3401,225,48,167,810,2697,1001,797,730,536,610
8,9,6,6,0,0,1,2,3,0,0,2,0,4
9,10,1103,998,24,81,63,310,730,188,157,211,222,325


In [63]:
# Para facilitar el uso de la función join agregamos una columna con solo la información del año
c = []
for i in range(0, len(df_budget)):
    y = df_budget.loc[i, 'Year'].year
    c.append(y)

df_budget['Year'] = c

# Creamos una copia del dataframe df_budget
df_budget_sum = df_budget.copy()
df_budget_sum['Year'] = df_budget_sum['Year'].astype(str)
df_budget_sum

/var/folders/r2/wrdzf10d2lq1sjgp521xsvrm0000gn/T/ipykernel_10689/633209279.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_budget['Year'] = c


,State ID,State,Year,Amount Approved,ANIO
0,1,Aguascalientes,2013,1511908040,2013
1,1,Aguascalientes,2014,1865135779,2014
2,1,Aguascalientes,2015,1659723400,2015
3,1,Aguascalientes,2016,1717532284,2016
4,1,Aguascalientes,2017,1230564370,2017
...,...,...,...,...,...
283,32,Zacatecas,2017,1992505733,2017
284,32,Zacatecas,2018,2284882371,2018
285,32,Zacatecas,2019,2567673138,2019
286,32,Zacatecas,2020,2010497486,2020


In [65]:
# Abrimos el diccionario de entidades

# Generamos el dataframe de entidades
entidades = pd.read_csv("tc_entidad.csv", index_col=False)

# Cambiamos los valores de nombres de entidad por los nombres cortos
entidades.replace('Veracruz de Ignacio de la Llave', 'Veracruz', True)
entidades.replace('Michoacan de Ocampo', 'Michoacan', True)
entidades.replace('Coahuila de Zaragoza', 'Coahuila', True)

entidades

,ID_ENTIDAD,NOM_ENTIDAD
0,1,Aguascalientes
1,2,Baja California
2,3,Baja California Sur
3,4,Campeche
4,5,Coahuila
5,6,Colima
6,7,Chiapas
7,8,Chihuahua
8,9,Ciudad de México
9,10,Durango


In [66]:
# Reacomodamos df_budget_sum de long a wide
df_budget_sum = pd.pivot(df_budget_sum, index='State ID', columns='Year', values='Amount Approved')
df_budget_sum

Year,2013,2014,2015,2016,2017,2018,2019,2020,2021
State ID,,,,,,,,,
1,1511908040,1865135779,1659723400,1717532284,1230564370,763497156,698139978,801262825,619602783
2,2558020712,4709676663,2494693399,2685756585,1570031154,9730725095,6664542845,1443654047,2622630589
3,2239023615,1941923516,2032543717,2855727063,1201590701,2032211730,1474567507,1102611193,937524627
4,99082029192,110330787664,179996903236,130369164237,105920604960,92634515442,89364091716,167022332994,170554682448
5,2402936855,2950138752,3973057572,3560035868,1764773023,2094854701,11314959253,5078344488,1812974402
6,2955366382,2860224449,1944919437,981280851,740663349,898703113,880445659,628087453,778436351
7,18648463351,19722914503,21161352210,20375385079,18862545588,16857348106,16724549508,19883055289,24557014449
8,4135868390,3522384746,5057660429,4396486205,2721150620,2987529745,2859344186,2479778252,2678139066
9,64223856150,86507293811,72355571523,63282614015,42290225860,37346399033,75528167216,33643021494,32943680506


In [67]:
#format column names
df_budget_sum.columns = ['Presupuesto '+ col for col in df_budget_sum.columns]

#reset index
df_budget_sum.reset_index(inplace=True)

#view updated pivot table
df_budget_sum

,State ID,Presupuesto 2013,Presupuesto 2014,Presupuesto 2015,Presupuesto 2016,Presupuesto 2017,Presupuesto 2018,Presupuesto 2019,Presupuesto 2020,Presupuesto 2021
0,1,1511908040,1865135779,1659723400,1717532284,1230564370,763497156,698139978,801262825,619602783
1,2,2558020712,4709676663,2494693399,2685756585,1570031154,9730725095,6664542845,1443654047,2622630589
2,3,2239023615,1941923516,2032543717,2855727063,1201590701,2032211730,1474567507,1102611193,937524627
3,4,99082029192,110330787664,179996903236,130369164237,105920604960,92634515442,89364091716,167022332994,170554682448
4,5,2402936855,2950138752,3973057572,3560035868,1764773023,2094854701,11314959253,5078344488,1812974402
5,6,2955366382,2860224449,1944919437,981280851,740663349,898703113,880445659,628087453,778436351
6,7,18648463351,19722914503,21161352210,20375385079,18862545588,16857348106,16724549508,19883055289,24557014449
7,8,4135868390,3522384746,5057660429,4396486205,2721150620,2987529745,2859344186,2479778252,2678139066
8,9,64223856150,86507293811,72355571523,63282614015,42290225860,37346399033,75528167216,33643021494,32943680506
9,10,3294028743,4698276429,4518541956,3666945990,3629190928,2524266459,2744707097,2576443203,3919983944


Agregamos el nombre de los estados

In [68]:
# Unimos los dataframes mediante la función join para formar nuestro dataframe suma
df_budget_sum = df_budget_sum.join(entidades.set_index('ID_ENTIDAD'), on='State ID')
df_budget_sum

,State ID,Presupuesto 2013,Presupuesto 2014,Presupuesto 2015,Presupuesto 2016,Presupuesto 2017,Presupuesto 2018,Presupuesto 2019,Presupuesto 2020,Presupuesto 2021,NOM_ENTIDAD
0,1,1511908040,1865135779,1659723400,1717532284,1230564370,763497156,698139978,801262825,619602783,Aguascalientes
1,2,2558020712,4709676663,2494693399,2685756585,1570031154,9730725095,6664542845,1443654047,2622630589,Baja California
2,3,2239023615,1941923516,2032543717,2855727063,1201590701,2032211730,1474567507,1102611193,937524627,Baja California Sur
3,4,99082029192,110330787664,179996903236,130369164237,105920604960,92634515442,89364091716,167022332994,170554682448,Campeche
4,5,2402936855,2950138752,3973057572,3560035868,1764773023,2094854701,11314959253,5078344488,1812974402,Coahuila
5,6,2955366382,2860224449,1944919437,981280851,740663349,898703113,880445659,628087453,778436351,Colima
6,7,18648463351,19722914503,21161352210,20375385079,18862545588,16857348106,16724549508,19883055289,24557014449,Chiapas
7,8,4135868390,3522384746,5057660429,4396486205,2721150620,2987529745,2859344186,2479778252,2678139066,Chihuahua
8,9,64223856150,86507293811,72355571523,63282614015,42290225860,37346399033,75528167216,33643021494,32943680506,Ciudad de México
9,10,3294028743,4698276429,4518541956,3666945990,3629190928,2524266459,2744707097,2576443203,3919983944,Durango


Agregamos la información sobre el presupuesto estatal aprobado anualmente en Sonora al dataframe con la suma de accidentes.

In [69]:
# Unimos los dataframes mediante la función join para formar nuestro dataframe suma_estatal
tidy_suma = tidy_suma.join(df_budget_sum.set_index('State ID'), on='ID_ENTIDAD')
tidy_suma

,ID_ENTIDAD,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,ANIO_2017,ANIO_2018,...,Presupuesto 2013,Presupuesto 2014,Presupuesto 2015,Presupuesto 2016,Presupuesto 2017,Presupuesto 2018,Presupuesto 2019,Presupuesto 2020,Presupuesto 2021,NOM_ENTIDAD
0,1,1222,1192,30,0,86,440,696,228,241,...,1511908040,1865135779,1659723400,1717532284,1230564370,763497156,698139978,801262825,619602783,Aguascalientes
1,2,489,479,1,9,19,128,342,14,60,...,2558020712,4709676663,2494693399,2685756585,1570031154,9730725095,6664542845,1443654047,2622630589,Baja California
2,3,93,87,5,1,4,30,59,21,21,...,2239023615,1941923516,2032543717,2855727063,1201590701,2032211730,1474567507,1102611193,937524627,Baja California Sur
3,4,559,505,50,4,52,182,325,117,126,...,99082029192,110330787664,179996903236,130369164237,105920604960,92634515442,89364091716,167022332994,170554682448,Campeche
4,5,466,427,33,6,30,88,348,71,69,...,2402936855,2950138752,3973057572,3560035868,1764773023,2094854701,11314959253,5078344488,1812974402,Coahuila
5,6,1852,1655,142,55,42,650,1160,365,360,...,2955366382,2860224449,1944919437,981280851,740663349,898703113,880445659,628087453,778436351,Colima
6,7,2492,2485,5,2,214,674,1604,415,457,...,18648463351,19722914503,21161352210,20375385079,18862545588,16857348106,16724549508,19883055289,24557014449,Chiapas
7,8,3674,3401,225,48,167,810,2697,1001,797,...,4135868390,3522384746,5057660429,4396486205,2721150620,2987529745,2859344186,2479778252,2678139066,Chihuahua
8,9,6,6,0,0,1,2,3,0,0,...,64223856150,86507293811,72355571523,63282614015,42290225860,37346399033,75528167216,33643021494,32943680506,Ciudad de México
9,10,1103,998,24,81,63,310,730,188,157,...,3294028743,4698276429,4518541956,3666945990,3629190928,2524266459,2744707097,2576443203,3919983944,Durango


In [70]:
tidy_suma.describe()

,ID_ENTIDAD,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,ANIO_2017,ANIO_2018,...,ANIO_2021,Presupuesto 2013,Presupuesto 2014,Presupuesto 2015,Presupuesto 2016,Presupuesto 2017,Presupuesto 2018,Presupuesto 2019,Presupuesto 2020,Presupuesto 2021
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,...,31.000000,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01
mean,16.000000,2579.322581,2465.290323,73.032258,41.000000,130.129032,680.548387,1768.645161,470.548387,555.258065,...,553.806452,1.562030e+10,1.828867e+10,1.898952e+10,1.587426e+10,1.209755e+10,1.200207e+10,1.526287e+10,1.468929e+10,1.653421e+10
std,9.092121,2249.986998,2128.050472,99.938809,92.266643,120.804178,635.701756,1651.660913,382.583572,588.889858,...,491.106738,2.819255e+10,3.302041e+10,3.655998e+10,2.660047e+10,2.112487e+10,1.971881e+10,2.834057e+10,3.287011e+10,3.640817e+10
min,1.000000,6.000000,6.000000,0.000000,0.000000,1.000000,2.000000,3.000000,0.000000,0.000000,...,4.000000,1.511908e+09,1.810654e+09,1.659723e+09,9.812809e+08,7.406633e+08,7.634972e+08,6.981400e+08,6.280875e+08,6.196028e+08
25%,8.500000,1033.000000,976.000000,10.500000,3.500000,46.000000,275.500000,697.500000,178.000000,192.500000,...,242.000000,2.839353e+09,3.236262e+09,3.926220e+09,3.207881e+09,1.751224e+09,2.117837e+09,2.351903e+09,1.895745e+09,2.382663e+09
50%,16.000000,1852.000000,1679.000000,33.000000,9.000000,86.000000,483.000000,1160.000000,407.000000,434.000000,...,354.000000,5.537671e+09,5.275054e+09,6.029125e+09,5.222430e+09,4.083207e+09,3.859308e+09,4.659184e+09,3.909489e+09,3.919984e+09
75%,23.500000,3557.500000,3357.000000,91.000000,33.000000,176.500000,742.000000,2658.500000,640.000000,663.000000,...,711.500000,9.786853e+09,1.298824e+10,1.094421e+10,1.146026e+10,9.885458e+09,9.687041e+09,1.053563e+10,8.973088e+09,1.141823e+10
max,31.000000,8417.000000,7804.000000,511.000000,450.000000,432.000000,2576.000000,6263.000000,1623.000000,2682.000000,...,1845.000000,1.142116e+11,1.359797e+11,1.799969e+11,1.303692e+11,1.059206e+11,9.263452e+10,1.216967e+11,1.670223e+11,1.705547e+11


In [71]:
tidy_suma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ID_ENTIDAD                           31 non-null     int64 
 1   Total_accidentes                     31 non-null     int64 
 2   CAUSAACCI_Conductor                  31 non-null     int64 
 3   CAUSAACCI_Mala condición del camino  31 non-null     int64 
 4   CAUSAACCI_Otra                       31 non-null     int64 
 5   CLASACC_Fatal                        31 non-null     int64 
 6   CLASACC_No fatal                     31 non-null     int64 
 7   CLASACC_Sólo daños                   31 non-null     int64 
 8   ANIO_2017                            31 non-null     int64 
 9   ANIO_2018                            31 non-null     int64 
 10  ANIO_2019                            31 non-null     int64 
 11  ANIO_2020                            31 non-nul

In [72]:
col = ['Presupuesto 2013', 'Presupuesto 2014', 'Presupuesto 2015', 'Presupuesto 2016']
for x in col:
    tidy_suma.pop(x)

In [73]:
tidy_suma.rename(columns = {'NOM_ENTIDAD':'Entidad'}, inplace = True)
tidy_suma

,ID_ENTIDAD,Total_accidentes,CAUSAACCI_Conductor,CAUSAACCI_Mala condición del camino,CAUSAACCI_Otra,CLASACC_Fatal,CLASACC_No fatal,CLASACC_Sólo daños,ANIO_2017,ANIO_2018,ANIO_2019,ANIO_2020,ANIO_2021,Presupuesto 2017,Presupuesto 2018,Presupuesto 2019,Presupuesto 2020,Presupuesto 2021,Entidad
0,1,1222,1192,30,0,86,440,696,228,241,191,254,308,1230564370,763497156,698139978,801262825,619602783,Aguascalientes
1,2,489,479,1,9,19,128,342,14,60,74,123,218,1570031154,9730725095,6664542845,1443654047,2622630589,Baja California
2,3,93,87,5,1,4,30,59,21,21,18,15,18,1201590701,2032211730,1474567507,1102611193,937524627,Baja California Sur
3,4,559,505,50,4,52,182,325,117,126,121,97,98,105920604960,92634515442,89364091716,167022332994,170554682448,Campeche
4,5,466,427,33,6,30,88,348,71,69,105,77,144,1764773023,2094854701,11314959253,5078344488,1812974402,Coahuila
5,6,1852,1655,142,55,42,650,1160,365,360,355,387,385,740663349,898703113,880445659,628087453,778436351,Colima
6,7,2492,2485,5,2,214,674,1604,415,457,613,448,559,18862545588,16857348106,16724549508,19883055289,24557014449,Chiapas
7,8,3674,3401,225,48,167,810,2697,1001,797,730,536,610,2721150620,2987529745,2859344186,2479778252,2678139066,Chihuahua
8,9,6,6,0,0,1,2,3,0,0,2,0,4,42290225860,37346399033,75528167216,33643021494,32943680506,Ciudad de México
9,10,1103,998,24,81,63,310,730,188,157,211,222,325,3629190928,2524266459,2744707097,2576443203,3919983944,Durango


### 3.3 Guardamos el dataframe generado como tidy_suma.csv

In [74]:
# Guardamos el data frame en un csv
tidy_suma.to_csv('tidy_suma.csv', header=True, index=False)